In [1]:
from src.models import hmm, decoders
import numpy as np

# Iterating through test use case lecture

In [2]:
# Original implementation
# index annotation observation_states=[i,j]    
observation_states = ['committed','ambivalent'] # A graduate student's dedication to their rotation lab

# index annotation hidden_states=[i,j]
hidden_states = ['R01','R21'] # The NIH funding source of the graduate student's rotation project 

# PONDERING QUESTION: How would a user define/compute their own HMM instantiation inputs to decode the hidden states for their use case observations?
use_case_one_data = np.load('./data/UserCase-Lecture.npz')

# Instantiate submodule class models.HiddenMarkovModel with
# observation and hidden states and prior, transition, and emission probabilities.
use_case_one_hmm = hmm.HiddenMarkovModel(observation_states,
                                     hidden_states,
                  use_case_one_data['prior_probabilities'], # prior probabilities of hidden states in the order specified in the hidden_states list
                  use_case_one_data['transition_probabilities'], # transition_probabilities[:,hidden_states[i]]
                  use_case_one_data['emission_probabilities']) # emission_probabilities[hidden_states[i],:][:,observation_states[j]]

# Instantiate submodule class models.ViterbiAlgorithm using the use case one HMM 
use_case_one_viterbi = decoders.ViterbiAlgorithm(use_case_one_hmm)

In [3]:
# Pull out information
observation_states = use_case_one_hmm.observation_states
print("Observation states")
print(observation_states)
observation_states_dict = use_case_one_hmm.observation_states_dict
hidden_states = use_case_one_hmm.hidden_states
print("Hidden states")
print(hidden_states)
hidden_states_dict = use_case_one_hmm.hidden_states_dict

prior_probs = use_case_one_hmm.prior_probabilities
transition_probs = use_case_one_hmm.transition_probabilities
emission_probs = use_case_one_hmm.emission_probabilities

print("Prior Probs")
print(prior_probs)
print("Transition Probs")
print(transition_probs)
print("Emission Probs")
print(emission_probs)

decode_observation_states = use_case_one_data['observation_states']

Observation states
['committed', 'ambivalent']
Hidden states
['R01', 'R21']
Prior Probs
[0.67 0.33]
Transition Probs
[[0.8 0.2]
 [0.4 0.6]]
Emission Probs
[[0.8 0.2]
 [0.4 0.6]]


In [4]:
num_hidden = len(hidden_states)    # Number of hidden states
num_obs = len(decode_observation_states)  # Length of observation sequence

# Initialize D and E matrices
all_deltas = np.zeros((num_hidden, num_obs)) # all deltas
paths = np.zeros((num_hidden, num_obs))
paths[:,0] = [hidden_state_index for hidden_state_index in range(len(hidden_states))]
all_deltas[:, 0] = np.multiply(prior_probs, emission_probs[:, observation_states_dict[decode_observation_states[0]]])

# Compute D and E in a nested loop
for trellis_node in range(1, num_obs):
    current_observation_state = observation_states_dict[decode_observation_states[trellis_node]]
    for _hidden_state in range(0,num_hidden):
        delta_trans = np.multiply(transition_probs[:, _hidden_state], all_deltas[:, trellis_node-1])
        new_delta = np.max(delta_trans) * emission_probs[_hidden_state, current_observation_state]
        all_deltas[_hidden_state, trellis_node] = new_delta
        paths[_hidden_state, trellis_node] = np.argmax(delta_trans)

In [5]:
# Back trace
best_path = np.zeros(num_obs).astype(np.int32)
# Get the index of the final delta
best_path[num_obs-1] = np.argmax(all_deltas[:, num_obs-1])
for n in range(num_obs-1, -1, -1):
    print(n)
    best_path[n-1] = paths[int(best_path[n]), n]

# Convert best_path into words
hidden_state_path = np.array([hidden_states_dict[i] for i in best_path])

5
4
3
2
1
0


In [6]:
print("Best path")
print(best_path)
print("Hidden state path")
print(hidden_state_path)

print(use_case_one_data['hidden_states'])

Best path
[0 0 1 1 1 0]
Hidden state path
['R01' 'R01' 'R21' 'R21' 'R21' 'R01']
['R01' 'R01' 'R21' 'R21' 'R21' 'R01']
